In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.metrics import accuracy_score

In [3]:
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf

In [5]:
X_train = []
Y_train = []
image_size = 150
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
for i in labels:
    folderPath = os.path.join('C:/Users/CHINMAYEE/Brain Tumor Classification/Dataset/Training',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('C:/Users/CHINMAYEE/Brain Tumor Classification/Dataset/Testing',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [6]:
X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
X_train.shape

(3264, 150, 150, 3)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 101)

ValueError: Found input variables with inconsistent numbers of samples: [2937, 3264]

In [9]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [10]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu', input_shape=(150,150,3)))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(Conv2D(64,(3,3),activation = 'relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation = 'relu'))
model.add(Conv2D(256,(3,3),activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(4,activation = 'softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 69, 69, 64)        0

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'] )

In [13]:
history = model.fit(X_train, y_train, epochs=30, validation_split=0.1)

Epoch 1/30
83/83 [==============================] - 240s 3s/step - loss: 2.2256 - accuracy: 0.3076 - val_loss: 1.3405 - val_accuracy: 0.4456
Epoch 2/30
83/83 [==============================] - 222s 3s/step - loss: 1.1162 - accuracy: 0.5112 - val_loss: 1.0256 - val_accuracy: 0.5510
Epoch 3/30
83/83 [==============================] - 231s 3s/step - loss: 0.9736 - accuracy: 0.5974 - val_loss: 0.9755 - val_accuracy: 0.5272
Epoch 4/30
83/83 [==============================] - 237s 3s/step - loss: 0.7998 - accuracy: 0.6568 - val_loss: 0.7893 - val_accuracy: 0.6599
Epoch 5/30
83/83 [==============================] - 231s 3s/step - loss: 0.6917 - accuracy: 0.7106 - val_loss: 0.8394 - val_accuracy: 0.6429
Epoch 6/30
83/83 [==============================] - 229s 3s/step - loss: 0.6281 - accuracy: 0.7450 - val_loss: 0.7544 - val_accuracy: 0.6361
Epoch 7/30
83/83 [==============================] - 231s 3s/step - loss: 0.5326 - accuracy: 0.7855 - val_loss: 0.5901 - val_accuracy: 0.7449
Epoch 8/30
83

In [15]:
model.save('braintumor.h5')